# Reinforcement Learning in Finite MDPs

In [0]:
!git clone https://github.com/rlgammazero/mvarl_hands_on.git > /dev/null 2>&1

## MDPs

In [0]:
import sys
sys.path.insert(0, './mvarl_hands_on/utils')
import numpy as np
from scipy.special import softmax # for SARSA
import matplotlib.pyplot as plt
import json
import math
from cliffwalk import CliffWalk

Setting up the environment

In [0]:
env = CliffWalk(proba_succ=0.98)

####################################################################################
# You probably want to test smaller enviroments before
# env = ToyEnv1(gamma=0.95)
####################################################################################

# Useful attributes
print("Set of states:", env.states)
print("Set of actions:", env.actions)
print("Number of states: ", env.Ns)
print("Number of actions: ", env.Na)
print("P has shape: ", env.P.shape)  # P[s, a, s'] = env.P[s, a, s']
print("discount factor: ", env.gamma)
print("")

# Usefult methods
state = env.reset() # get initial state
print("initial state: ", state)
print("reward at (s=1, a=3,s'=2): ", env.reward_func(1,3,2))
print("")

# A random policy
policy = np.random.randint(env.Na, size = (env.Ns,))
print("random policy = ", policy)

# Interacting with the environment
print("(s, a, s', r):")
for time in range(4):
    action = policy[state]
    next_state, reward, done, info = env.step(action)
    print(state, action, next_state, reward)
    if done:
        break
    state = next_state
print("")
print(env.R.shape)

## Question 1: Value iteration
1. Write a function applying the optimal Bellman operator on a provided Q function: $Q_1 = LQ_0, \; Q_0\in \mathbb{R}^{S\times A}$
2. Write a function implementing Value Iteration (VI) with $\infty$-norm stopping condition (reuse function implemented in 1)
3. Evaluate the convergence of your estimate, i.e., plot the value $\|V_n - V^\star\|_{\infty}$

In [0]:
# --------------
# Point 1
# --------------
def bellman_operator(Q0, Ns, Na, R, P,gamma):
    Q_max = np.max(Q0, axis=1)
    greedy_policy = np.argmax(Q0, axis=1)
    Q1 = np.asarray([[np.dot(P[state,action,:],(R[state,action, :] +gamma*Q_max)) for action in range(Na)] for state in range(Ns)])
    return Q1, greedy_policy

In [0]:
# --------------
# Point 2
# --------------
def value_iteration(Q0, env, epsilon=1e-5):
    # TODO (feel free to change the return argument)
    Q_history = [Q0]
    Q = Q0 
    start = True
    while (np.max( np.abs( Q - Q0 ) ) > epsilon or start):
        start = False
        Q0 = Q
        Q,greedy_policy= bellman_operator(Q0, env.Ns, env.Na, env.R, env.P, env.gamma)
        Q_history.append(Q)
    return Q, greedy_policy, Q_history

In [0]:
# --------------
# Point 3
# --------------
with open("./mvarl_hands_on/data/Q_opts.json", "r") as fp:
    Qopts = json.load(fp)
Qstar = np.array(Qopts["{}_{}".format(type(env).__name__,env.gamma)])
Q0 = np.zeros(Qstar.shape)
Q, greedy_policy, Q_history = value_iteration(Q0,env)

V_star = np.max(Qstar, axis=1)
norm_values = [np.max(np.abs(np.max(Q, axis=1)-V_star)) for Q in Q_history]
plt.plot(norm_values)
plt.xlabel('Iteration')
plt.ylabel('Error')
plt.title("Q-learning: Convergence of Q")

In [0]:
state = env.reset()
env.render()
for i in range(50):
    action = greedy_policy[state]
    state, reward, done, _ = env.step(action)
    env.render()

## Question 2: Q learning
Q learning is a model-free algorithm for estimating the optimal Q-function online.
It is an off-policy algorithm since the samples are collected with a policy that is (potentially) not the one associated to the estimated Q-function.

1. Implement Q learning with $\epsilon$-greedy exploration.
  - Plot the error in Q-functions over iterations
  - Plot the sum of rewards as a function of iteration


$\epsilon$-greedy policy:
$$
\pi(s) = \begin{cases}
\max_a Q(s,a) & \text{w.p.} \epsilon\\
\text{random action} & \text{w.p.} 1- \epsilon
\end{cases}
$$

In [0]:
# ---------------------------
# Q-Learning
# ---------------------------
# suggested interface
# you can change it!
import random
class QLearning:
    """
    Q learning with epsilon-greedy exploration
    """
    def __init__(self,Q0,gamma = env.gamma,eps = 0.15):
      self.gamma= gamma
      self.dim_Na = Q0.shape[1]
      self.dim_Ns = Q0.shape[0]
      self.eps = eps
      self.Q = Q0
      self.pi = np.argmax(Q0,axis =1)
      self.cumulative_rewards=[0]
    
    def sample_action(self, state, greedy):
      """ returns an action (int) following the eps-greedy policy"""
      rand_sample = np.random.random()
      
      if(rand_sample > self.eps ):
        # generate random action
        action = np.random.randint(0,self.dim_Na)
      else :
        # Take max action 
        action = greedy[state]
      return action
    
    def update(self, state, action, next_state, reward, alpha):
      """performs an update of the state action value function"""
      delta = reward + self.gamma*(self.Q[next_state]).max() - self.Q[state,action] 
      self.Q[state,action] = self.Q[state,action]+alpha*delta
      self.pi_update()
      self.cumulative_rewards.append(self.cumulative_rewards[-1]*self.gamma + reward)
    def pi_update(self):
      self.pi = np.argmax(self.Q, axis =1)

    

In [0]:
# --------------
# Point 1
# --------------
# Number of Q learning steps
max_steps = int(1e4)  
# max_steps = 10

Q0 = np.zeros((env.Ns, env.Na))
# Use the previous code to verify the correctness of q learning
Q_opt, pi_opt,Q_hist = value_iteration(Q0, env, epsilon=1e-8)

#------------------------------------------------------------------------------#

ql = QLearning(Q0,env.gamma)
# main algorithmic loop
norm_values = []
#obs = env.reset()
num_runs = 100
visits = np.zeros((env.Ns,env.Na))+1
cum_r = []
for i in range(num_runs):
  obs = env.reset() 
  done = False
  t = 0
  obs = env.reset()
  while t < max_steps and not done:

    action = ql.sample_action(obs,ql.pi)
    observation, reward, done, info = env.step(action)
    norm_values.append(np.abs(ql.Q - Q_opt).mean())
    # Perform update for Q
    ql.update(obs,action, observation, reward,1/(i+1))
    visits[obs,action]+=1
    t = t + 1
    obs = observation
    
  cum_r.append(np.sum(ql.cumulative_rewards))
  ql.cumulative_rewards =[0]
print(env.render())
print("optimal policy: ", pi_opt)
greedy_policy = np.argmax(ql.Q, axis=1)
print("est policy:", greedy_policy)
 

f, (ax1, ax2) = plt.subplots(1, 2, figsize = (10,5))

ax1.plot(norm_values)
ax1.set_xlabel('Iteration')
ax1.set_ylabel('Error')
ax1.set_title("Q-learning: Convergence of Q")
ax2.plot(cum_r)
ax2.set_xlabel('Iteration')
ax2.set_ylabel('rewards')
ax2.set_title("Cumulative sum of rewards through Q-learning ")

# how confident are you in the performance of the algorithm? maybe a single run is not enough

---
## Comments
>Q-learning algo seems to have difficulty to converge with this setting